<a href="https://colab.research.google.com/github/Schwaldlander/nanogpt_demo/blob/main/train_gpt_kaggle_run_tiny_stories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!huggingface-cli cli

Token is valid (permission: fineGrained).
The token `Sevres` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Sevres`


In [ ]:
!pip install -U "datasets>=2.15" "huggingface_hub[fsspec]" fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 76.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.4/515.4 kB 22.3 MB/s eta 0:00:00
  Attempting uninstall: hf-xet
    Found existing installation: hf-xet 1.1.0
    Uninstalling hf-xet-1.1.0:
      Successfully uninstalled hf-xet-1.1.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.31.1
    Uninstalling huggingface-hub-0.31.1:
      Successfully uninstalled huggingface-hub-0.31.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is in

In [ ]:
# -*- coding: utf-8 -*-
"""train_gpt2_colab.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/15XnZcXnQr0K4W3QmiraXiHSue-FcHE69

# Fine‑tune GPT‑2 from scratch on wikitext //EDU FineWeb‑10B

This Colab notebook refactors **`train_gpt2.py`** (from https://github.com/karpathy/build-nanogpt/blob/master/train_gpt2.py) into a step‑by‑step workflow. Releant explanation can be found in [https://www.youtube.com/watch?v=l8pRSuU81PU]:

1. **Install** required libraries
2. **Define** GPT‑2 building blocks
3. **Prepare** an ultra‑lightweight streaming dataloader
4. **Configure** (optional) Distributed Data Parallel (DDP)
5. **Train**, **validate** on HellaSwag, and **sample** text

Feel free to tweak hyper‑parameters such as `max_steps` and `total_batch_size` to match your compute budget.
"""

# @title Install dependencies
# !pip install -q tiktoken

# !python -c "import fsspec; fsspec.filesystem('hf')"

# !pip install -U "datasets>=2.15" "huggingface_hub[fsspec]" fsspec

# !huggingface-cli whoami

# !huggingface-cli login

"""## Imports

hellasawg library is a challenge dataset of filling sentences. Project Link[https://huggingface.co/datasets/Rowan/hellaswag]
"""
import re
import os, math, time, inspect
from dataclasses import dataclass

import torch
import torch.nn as nn
from torch.nn import functional as F

import tiktoken
import numpy as np
# from hellaswag import render_example, iterate_examples
import datasets
import torch, itertools
from datasets import load_dataset
from transformers import AutoTokenizer

class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1
        # regularization
        self.n_head = config.n_head
        self.n_embd = config.n_embd

    def forward(self, x):
        B, T, C = x.size()
        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        # nh is "number of heads", hs is "head size", and C (number of channels) = nh * hs
        qkv = self.c_attn(x)
        # normally we don't define three separate linear layers, but project input to 3times dim
        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        y = F.scaled_dot_product_attention(q, k, v, is_causal=True)
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.c_proj(y)
        return y

"""## Transformer building blocks

Unlike ReLU, GELU returns non-zero values when $x<0$, given by

$\operatorname{GELU}(x) \;\approx\; 0.5 \, x \,\Bigl(1 + \tanh\!\Bigl[\sqrt{\tfrac{2}{\pi}}\,
  \bigl(x + 0.044715\,x^{3}\bigr)\Bigr]\Bigr)$

"""

class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc   = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu   = nn.GELU(approximate='tanh')
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

# Residual connection to control activation variance growth in the stream in forward propogation,
#and to preserve information and gradient flow, making very deep nets trainable
class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp  = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

"""## GPT‑2 model wrapper

n_embd is chosen following GPT2 paper.
vocab_size is chosen after GPT-2 tokenizer
As for the size of T, Do not try to finetune GPT-2 on longer sequences unless you are prepared to alter its architecture.

Note that LayerNorm is **after** Transformer modules to bring stabler gradients in very deep nets; allows cranking the learning-rate without warm-up gymnastics.

# FlashAttention
 consists of kernel fusion operation. It is high performance implementation of the attention mechanism in transformers. It is fast, memory-efficient and numerically stable, by reducing GPU I/O ops.

 1. Computes blocks of attention scores in SRAM GPU registers, instead of materializing large NxN attention matrix in High Bandwidth Memory (HBM)

 2. Use tiling and fused kernel operations to reduce memory reads/writes

 3. Applies softmax + dropout + matmal in one fused kernel, though inference paths usually compile without dropout for speed.



"""

@dataclass
class GPTConfig:
    block_size: int = 1024
    vocab_size: int = 12565#50257
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768


class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(
            dict(
                wte   = nn.Embedding(config.vocab_size, config.n_embd),
                wpe   = nn.Embedding(config.block_size, config.n_embd),
                h     = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
                ln_f  = nn.LayerNorm(config.n_embd),
            )
        )
        #use learnable position encoding
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        # weight sharing that enhances efficiency and proves effective
        self.transformer.wte.weight = self.lm_head.weight
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            if hasattr(module, 'NANOGPT_SCALE_INIT'):
                std *= (2 * self.config.n_layer) ** -0.5
            nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.size()
        assert T <= self.config.block_size, "Sequence length > block size"
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device)
        pos_emb = self.transformer.wpe(pos)
        tok_emb = self.transformer.wte(idx)
        x = tok_emb + pos_emb
        for block in self.transformer.h:
            x = block(x)
        # Notice that the third layernorm before lm_head
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

"""## Streaming dataloader & helpers

Note that not every dataset token is processed by GPT training. The vocab size is larger than actual size to make itself divisble by 2-exponents. Throughput is increased despite vocal_size raised.

Data Augementation(noise adding, shuffling) not added
"""

import torch
from datasets import load_dataset
from transformers import AutoTokenizer


class StreamingBatchLoader:
    """
    Replaces the original GPTDataset in nanoGPT for streaming use.
    Generates (x, y) token blocks from a streaming Hugging Face dataset.
    """
    def __init__(self, repo_id, version_name, split="train", block_size=1024, batch_size=8, seed=42):
        self.block_size = block_size
        self.batch_size = batch_size
        self.buf = []

        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True,model_max_length=10_000_000)
        self.tokenizer.pad_token = self.tokenizer.eos_token  # For safety

        # Load streaming dataset               name=version_name,
        if len(version_name)==0:
            ds = load_dataset(
            repo_id,

            split="train",
            streaming=True)
        else:
            ds = load_dataset(
            repo_id,
            version_name,
            split="train",
            streaming=True)
        ds = ds.shuffle(buffer_size=10_000, seed=seed)
        ds = ds.map(self._tokenize, remove_columns=["text"])
        self.iterator = iter(ds)

    _re_unk   = re.compile(r"\s*<unk>\s*")
    _re_at_at = re.compile(r"\s*@\s*")
    _re_head  = re.compile(r"={2,}")            # '== Heading =='

    def _clean(self, text: str) -> str:
        text = self._re_unk.sub(" ", text)
        text = (text.replace("@-@", "-")
                    .replace("@,@", ",")
                    .replace("@.@", "."))
        text = self._re_at_at.sub(" ", text)
        text = self._re_head.sub(" ", text)
        return " ".join(text.split())           # squeeze extra spaces

    def _tokenize(self, example):
        text = re.sub(r"\s*<unk>\s*", " ", example["text"])
        # Replace "@-@", "@,@", etc. with plain punctuation
        text = text.replace("@-@", "-").replace("@,@", ",").replace("@.@", ".")
        # Remove stray standalone "@"
        text = re.sub(r"\s*@\s*", " ", text)
        ids = self.tokenizer.encode(text.replace("={2,}", " "))   # strip wiki '=='
        return {"ids": ids + [self.tokenizer.eos_token_id]}

    def next(self):
        """
        Returns one (x, y) batch of shape (B, T)
        """
        required = self.batch_size * self.block_size + 1
        while len(self.buf) < required:
            self.buf.extend(next(self.iterator)["ids"])

        print(len(self.buf))
        tokens = torch.tensor(self.buf[:required], dtype=torch.long)
        print(tokens.shape)
        del self.buf[:self.batch_size * self.block_size]

        x = tokens[:-1].view(self.batch_size, self.block_size)
        y = tokens[1:].view(self.batch_size, self.block_size)
        if x.shape != (self.batch_size, self.block_size):
            print('x shape error')
            x = None
        if y.shape != (self.batch_size, self.block_size):
            print('y shape error')
            y = None
        return x, y
    def humanize(self, tensor_batch):
        """
        tensor_batch: (B, T) on *CPU*.
        Returns readable words list[str] of length B.
        """
        return [
            self.tokenizer.decode(row.tolist(), skip_special_tokens=True)
            for row in tensor_batch
        ]

# Hyper‑parameters (tweak to taste)
total_batch_size = 524288  # 2**19, approx 0.5M, in number of tokens for Fineweb edu
total_batch_size = 8192 # for WikiText
B = 8
T = 1024

max_lr   = 6e-4
min_lr   = max_lr * 0.1
warmup_steps = 715
max_steps  = 3200  # shorter default for Colab; raise for full training

train_loader = StreamingBatchLoader(repo_id="roneneldan/TinyStories", version_name="",split="train", block_size=B, batch_size=T)
x, y = train_loader.next()
print(x.shape, y.shape)

"""## Device / DDP setup & hyper‑parameters

Distributed Data Parallel(DDP) saves GPU computation resources. Every GPU has its separate model and dataloader.

RANK = 0 means main process.
"""

# Detect distributed run
ddp = int(os.environ.get('RANK', -1)) != -1
if ddp:
    import torch.distributed as dist
    from torch.distributed import init_process_group
    from torch.nn.parallel import DistributedDataParallel as DDP

    init_process_group(backend='nccl')
    ddp_rank = int(os.environ['RANK'])
    ddp_local_rank = int(os.environ['LOCAL_RANK'])
    ddp_world_size = int(os.environ['WORLD_SIZE'])
    device = f'cuda:{ddp_local_rank}'
    torch.cuda.set_device(device)
    master_process = ddp_rank == 0
else:
    ddp_rank = 0
    ddp_world_size = 1
    master_process = True
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

device_type = 'cuda' if device.startswith('cuda') else 'cpu'
print('Using device:', device, 'rank', ddp_rank)

assert total_batch_size % (B * T * ddp_world_size) == 0
grad_accum_steps = total_batch_size // (B * T * ddp_world_size)
print('Gradient accumulation steps:', grad_accum_steps)



"""## Build model & dataloaders

Before reaching warmup iterations, learning rate increases linearly. After the set steps, cosine decay is applied.

In AdamW optimizer, decoupled weight decay is used as L2 regularization to penalize large weights and avoid overfitting. Compared to SGD, it reduces the ampitude of gradient flucations.

Aside from the code implementations, it is also feasible to dynamically increase batch size.
"""

model = GPT(GPTConfig())
model.to(device)
if ddp:
    model = DDP(model, device_ids=[ddp_local_rank])
raw_model = model.module if ddp else model

def get_lr(it):
    if it < warmup_steps:
        return max_lr * (it + 1) / warmup_steps
    if it > max_steps:
        return min_lr
    decay_ratio = (it - warmup_steps) / (max_steps - warmup_steps)
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    return min_lr + coeff * (max_lr - min_lr)

optimizer = torch.optim.AdamW(
    raw_model.parameters(),
    lr=max_lr, betas=(0.9, 0.95), eps=1e-8, weight_decay=0.1, fused=('fused' in inspect.signature(torch.optim.AdamW).parameters and device_type=='cuda')
)

"""## Training loop
Typically the tiktoken compress rate is 3:1

When we wish to increase batch size but constainted to memory size, we can accumulate gradient on multiple mini batches, and defer backward propogation until all mini batches in one batch are processed.

If model.require_backward_grad_sync=True, then DDP uses al-reduce operation to average gradients on all gpus, then send back to ensure all the gradients are synchornized.

**why autocast to bfloat16:**

16-bit numbers use half the memory of 32-bit. bfloat16 has the same exponent range as float32, but fewer mantissa bits, so it can represent very large/small numbers **without** underflow/overflow.
"""



8389
torch.Size([8193])
torch.Size([1024, 8]) torch.Size([1024, 8])
Using device: cuda rank 0
Gradient accumulation steps: 1


'## Training loop\nTypically the tiktoken compress rate is 3:1\n\nWhen we wish to increase batch size but constainted to memory size, we can accumulate gradient on multiple mini batches, and defer backward propogation until all mini batches in one batch are processed.\n\nIf model.require_backward_grad_sync=True, then DDP uses al-reduce operation to average gradients on all gpus, then send back to ensure all the gradients are synchornized.\n\n**why autocast to bfloat16:**\n\n16-bit numbers use half the memory of 32-bit. bfloat16 has the same exponent range as float32, but fewer mantissa bits, so it can represent very large/small numbers **without** underflow/overflow.\n'

In [ ]:
# loader = StreamingBatchLoader(batch_size=2)  # small smoke test
# x, y = loader.next()
#tinystory
print("x shape:", x.shape, "y shape:", y.shape)
print("--- x[0] as text ----------------")
print(train_loader.humanize(x))

x shape: torch.Size([1024, 8]) y shape: torch.Size([1024, 8])
--- x[0] as text ----------------
['Anna and Ben were going on a trip', ' with their mom and dad. They were', ' very excited to see new things and have', ' fun. They packed their luggage with their', ' clothes, toys and books. Anna had', ' a furry teddy bear that she loved', ' very much. She put it in her', ' backpack and hugged it.\n\n"', 'Are you ready to go?" Mom asked', '. "Yes, we are!" Anna', ' and Ben said. They got in the', ' car and drove to the airport. They', ' had to wait in a long line to', ' check in their luggage. Anna and Ben', ' were bored and started to run around and', ' make noise.\n\n"Anna,', ' Ben, behave!" Dad said. "', 'You have to be quiet and stay with', ' us. There are many people here and', ' we don\'t want to lose you."', ' Anna and Ben nodded, but they did', ' not listen. They saw a big cart', ' with many suitcases on it. They', ' thought it would be fun to climb on', ' it and pretend they were 

In [ ]:

enc = tiktoken.get_encoding('gpt2')

for step in range(max_steps):
    t0 = time.time()
    model.train()
    optimizer.zero_grad()
    loss_accum = 0.0

    for micro_step in range(grad_accum_steps):
        x, y = train_loader.next()
        if x is None or y is None:
            continue
        x, y = x.to(device), y.to(device)
        if ddp:
            model.require_backward_grad_sync = (micro_step == grad_accum_steps - 1)
        # increase throughput with bfloat16
        with torch.autocast(device_type=device_type, dtype=torch.bfloat16):
            logits, loss = model(x, y)
        (loss / grad_accum_steps).backward()
        loss_accum += loss.detach()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    lr = get_lr(step)
    for pg in optimizer.param_groups:
        pg['lr'] = lr
    optimizer.step() # updates parameters

    dt = time.time() - t0
    if master_process:
        print(f'step {step:04d} | loss {loss_accum.item():.4f} | lr {lr:.2e} | time {dt*1000:.0f} ms')
    if step%400==0:
        model.eval()
        prompt = "Our family have a picnic "
        tokens = torch.tensor(enc.encode(prompt), dtype=torch.long, device=device).unsqueeze(0)
        max_len = 32
        for _ in range(max_len - tokens.size(1)):
            with torch.no_grad(), torch.autocast(device_type=device_type, dtype=torch.bfloat16):
                logits, _ = model(tokens)
            probs = torch.softmax(logits[:, -1], dim=-1)
            idx = torch.multinomial(probs, num_samples=1)
            tokens = torch.cat([tokens, idx], dim=-1)

        print(enc.decode(tokens[0].tolist()))

"""## Quick top‑k sampling check

This displays the performance of code: how the LLM fills sentences
"""


8408
torch.Size([8193])
step 0000 | loss 10.9561 | lr 8.39e-07 | time 2130 ms
Our family have a picnic  Caucus Petersen Brees OECD NEC!"enarioate pensions Sacrament sweet"[ 149 Alliance UA grill efficientlyatern Becker specificity Blankgew proposals sap 156 noticeable
8249
torch.Size([8193])
step 0001 | loss 10.9061 | lr 1.68e-06 | time 100 ms
8573
torch.Size([8193])
step 0002 | loss 10.7663 | lr 2.52e-06 | time 105 ms
8228
torch.Size([8193])
step 0003 | loss 10.5910 | lr 3.36e-06 | time 100 ms
8281
torch.Size([8193])
step 0004 | loss 10.4203 | lr 4.20e-06 | time 101 ms
8575
torch.Size([8193])
step 0005 | loss 10.1956 | lr 5.03e-06 | time 101 ms
8227
torch.Size([8193])
step 0006 | loss 10.0194 | lr 5.87e-06 | time 99 ms
8199
torch.Size([8193])
step 0007 | loss 9.8450 | lr 6.71e-06 | time 100 ms
8275
torch.Size([8193])
step 0008 | loss 9.7389 | lr 7.55e-06 | time 101 ms
8308
torch.Size([8193])
step 0009 | loss 9.6318 | lr 8.39e-06 | time 101 ms
8442
torch.Size([8193])
step 0010 | loss 9

In [ ]:
model.eval()
prompt = "Hello, I'm a LLM,"
tokens = torch.tensor(enc.encode(prompt), dtype=torch.long, device=device).unsqueeze(0)
max_len = 64
for _ in range(max_len - tokens.size(1)):
    with torch.no_grad(), torch.autocast(device_type=device_type, dtype=torch.bfloat16):
        logits, _ = model(tokens)
    probs = torch.softmax(logits[:, -1], dim=-1)
    idx = torch.multinomial(probs, num_samples=1)
    tokens = torch.cat([tokens, idx], dim=-1)

print(enc.decode(tokens[0].tolist()))